In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import glob
import matplotlib.pyplot as plt
from keras import layers
from keras import Model
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
import os
from mlxtend.data import loadlocal_mnist

# load training data
X, y = loadlocal_mnist(
    images_path='../emnist/emnist-letters-train-images-idx3-ubyte', 
    labels_path='../emnist/emnist-letters-train-labels-idx1-ubyte')

X = X.reshape(124800, 28, 28, 1)
X = np.divide(X, 255.)
y = y-1

# load testing data
Xtest, ytest = loadlocal_mnist(
    images_path='../emnist/emnist-letters-test-images-idx3-ubyte', 
    labels_path='../emnist/emnist-letters-test-labels-idx1-ubyte')

Xtest = Xtest.reshape(20800, 28, 28, 1)
Xtest = np.divide(Xtest, 255.)
ytest = ytest-1

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 530001890406509276
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23285923840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1348658162190911174
 physical_device_desc: "device: 0, name: Tesla M40 24GB, pci bus id: 0000:01:00.0, compute capability: 5.2"
 xla_global_id: 416903419]

## Model 1: Same architecture as in-class CNN example

In [4]:
np.random.seed(4622)
tf.random.set_seed(4622)

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B

img_input = layers.Input(shape=(28,28,1))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

x = layers.Conv2D(16,3,activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

x = layers.Conv2D(32,3,activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

x = layers.Conv2D(64,3,activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers

x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units

x = layers.Dense(512, activation='relu')(x) # captures more non-neighbor and non-linear relationships

# Create output layer with a single node and sigmoid activation

output = layers.Dense(26, activation='softmax')(x)

# Create model:

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss, metrics=['accuracy'])

In [5]:
history = model.fit(X, y, epochs=1000, batch_size=31200, validation_data=(Xtest, ytest))

Epoch 1/1000
4/4 [==============================] - 5s 579ms/step - loss: 3.2494 - accuracy: 0.0659 - val_loss: 3.2314 - val_accuracy: 0.0837
Epoch 2/1000
4/4 [==============================] - 1s 305ms/step - loss: 3.2151 - accuracy: 0.1145 - val_loss: 3.1808 - val_accuracy: 0.1767
Epoch 3/1000
4/4 [==============================] - 1s 309ms/step - loss: 3.1482 - accuracy: 0.2107 - val_loss: 3.0814 - val_accuracy: 0.2649
Epoch 4/1000
4/4 [==============================] - 1s 301ms/step - loss: 3.0200 - accuracy: 0.3131 - val_loss: 2.8992 - val_accuracy: 0.3716
Epoch 5/1000
4/4 [==============================] - 1s 300ms/step - loss: 2.7978 - accuracy: 0.3820 - val_loss: 2.6136 - val_accuracy: 0.3933
Epoch 6/1000
4/4 [==============================] - 1s 306ms/step - loss: 2.4860 - accuracy: 0.4118 - val_loss: 2.2828 - val_accuracy: 0.4375
Epoch 7/1000
4/4 [==============================] - 1s 306ms/step - loss: 2.1668 - accuracy: 0.4451 - val_loss: 2.0019 - val_accuracy: 0.4603
Epoch 

4/4 [==============================] - 1s 299ms/step - loss: 0.3905 - accuracy: 0.8715 - val_loss: 0.4051 - val_accuracy: 0.8693
Epoch 116/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.3884 - accuracy: 0.8716 - val_loss: 0.4100 - val_accuracy: 0.8673
Epoch 117/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.3892 - accuracy: 0.8717 - val_loss: 0.4068 - val_accuracy: 0.8675
Epoch 118/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.3868 - accuracy: 0.8728 - val_loss: 0.4043 - val_accuracy: 0.8691
Epoch 119/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.3866 - accuracy: 0.8729 - val_loss: 0.4025 - val_accuracy: 0.8700
Epoch 120/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.3839 - accuracy: 0.8730 - val_loss: 0.4043 - val_accuracy: 0.8698
Epoch 121/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.3866 - accuracy: 0.8714 - val_loss: 0.4006 - val_accuracy: 0.8721
Epoch 1

Epoch 172/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.3300 - accuracy: 0.8897 - val_loss: 0.3619 - val_accuracy: 0.8830
Epoch 173/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.3274 - accuracy: 0.8914 - val_loss: 0.3599 - val_accuracy: 0.8848
Epoch 174/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.3270 - accuracy: 0.8916 - val_loss: 0.3600 - val_accuracy: 0.8835
Epoch 175/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.3263 - accuracy: 0.8909 - val_loss: 0.3569 - val_accuracy: 0.8842
Epoch 176/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.3242 - accuracy: 0.8920 - val_loss: 0.3551 - val_accuracy: 0.8864
Epoch 177/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.3241 - accuracy: 0.8917 - val_loss: 0.3565 - val_accuracy: 0.8845
Epoch 178/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.3235 - accuracy: 0.8922 - val_loss: 0.3552 - val_accuracy:

Epoch 229/1000
4/4 [==============================] - 1s 308ms/step - loss: 0.2947 - accuracy: 0.9009 - val_loss: 0.3339 - val_accuracy: 0.8916
Epoch 230/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.2937 - accuracy: 0.9014 - val_loss: 0.3333 - val_accuracy: 0.8919
Epoch 231/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.2929 - accuracy: 0.9015 - val_loss: 0.3375 - val_accuracy: 0.8907
Epoch 232/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.2935 - accuracy: 0.9010 - val_loss: 0.3350 - val_accuracy: 0.8912
Epoch 233/1000
4/4 [==============================] - 1s 309ms/step - loss: 0.2914 - accuracy: 0.9024 - val_loss: 0.3359 - val_accuracy: 0.8917
Epoch 234/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.2923 - accuracy: 0.9017 - val_loss: 0.3375 - val_accuracy: 0.8894
Epoch 235/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.2928 - accuracy: 0.9017 - val_loss: 0.3338 - val_accuracy:

Epoch 286/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.2735 - accuracy: 0.9073 - val_loss: 0.3205 - val_accuracy: 0.8965
Epoch 287/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.2711 - accuracy: 0.9083 - val_loss: 0.3208 - val_accuracy: 0.8963
Epoch 288/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.2703 - accuracy: 0.9089 - val_loss: 0.3227 - val_accuracy: 0.8963
Epoch 289/1000
4/4 [==============================] - 1s 310ms/step - loss: 0.2714 - accuracy: 0.9087 - val_loss: 0.3197 - val_accuracy: 0.8970
Epoch 290/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.2694 - accuracy: 0.9091 - val_loss: 0.3214 - val_accuracy: 0.8948
Epoch 291/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.2692 - accuracy: 0.9094 - val_loss: 0.3197 - val_accuracy: 0.8967
Epoch 292/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.2689 - accuracy: 0.9096 - val_loss: 0.3236 - val_accuracy:

...

Epoch 742/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.1803 - accuracy: 0.9368 - val_loss: 0.3047 - val_accuracy: 0.9051
Epoch 743/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1810 - accuracy: 0.9369 - val_loss: 0.3050 - val_accuracy: 0.9036
Epoch 744/1000
4/4 [==============================] - 1s 310ms/step - loss: 0.1798 - accuracy: 0.9370 - val_loss: 0.3031 - val_accuracy: 0.9049
Epoch 745/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.1793 - accuracy: 0.9375 - val_loss: 0.3074 - val_accuracy: 0.9031
Epoch 746/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.1807 - accuracy: 0.9374 - val_loss: 0.3083 - val_accuracy: 0.9012
Epoch 747/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1815 - accuracy: 0.9359 - val_loss: 0.3073 - val_accuracy: 0.9036
Epoch 748/1000
4/4 [==============================] - 1s 310ms/step - loss: 0.1817 - accuracy: 0.9364 - val_loss: 0.3139 - val_accuracy:

Epoch 799/1000
4/4 [==============================] - 1s 299ms/step - loss: 0.1759 - accuracy: 0.9380 - val_loss: 0.3157 - val_accuracy: 0.8989
Epoch 800/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.1796 - accuracy: 0.9359 - val_loss: 0.3137 - val_accuracy: 0.9038
Epoch 801/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1771 - accuracy: 0.9371 - val_loss: 0.3072 - val_accuracy: 0.9044
Epoch 802/1000
4/4 [==============================] - 1s 310ms/step - loss: 0.1762 - accuracy: 0.9375 - val_loss: 0.3141 - val_accuracy: 0.9002
Epoch 803/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.1782 - accuracy: 0.9367 - val_loss: 0.3105 - val_accuracy: 0.9038
Epoch 804/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1758 - accuracy: 0.9380 - val_loss: 0.3096 - val_accuracy: 0.9039
Epoch 805/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.1761 - accuracy: 0.9378 - val_loss: 0.3163 - val_accuracy:

Epoch 856/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1667 - accuracy: 0.9412 - val_loss: 0.3078 - val_accuracy: 0.9039
Epoch 857/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.1661 - accuracy: 0.9419 - val_loss: 0.3074 - val_accuracy: 0.9040
Epoch 858/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1655 - accuracy: 0.9419 - val_loss: 0.3083 - val_accuracy: 0.9048
Epoch 859/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1652 - accuracy: 0.9424 - val_loss: 0.3086 - val_accuracy: 0.9050
Epoch 860/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1667 - accuracy: 0.9418 - val_loss: 0.3101 - val_accuracy: 0.9039
Epoch 861/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1691 - accuracy: 0.9407 - val_loss: 0.3121 - val_accuracy: 0.9026
Epoch 862/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1688 - accuracy: 0.9404 - val_loss: 0.3157 - val_accuracy:

Epoch 913/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.1633 - accuracy: 0.9422 - val_loss: 0.3132 - val_accuracy: 0.9026
Epoch 914/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1628 - accuracy: 0.9426 - val_loss: 0.3177 - val_accuracy: 0.9024
Epoch 915/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.1627 - accuracy: 0.9424 - val_loss: 0.3107 - val_accuracy: 0.9053
Epoch 916/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.1613 - accuracy: 0.9430 - val_loss: 0.3116 - val_accuracy: 0.9041
Epoch 917/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.1624 - accuracy: 0.9425 - val_loss: 0.3102 - val_accuracy: 0.9039
Epoch 918/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1612 - accuracy: 0.9433 - val_loss: 0.3099 - val_accuracy: 0.9042
Epoch 919/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.1601 - accuracy: 0.9441 - val_loss: 0.3085 - val_accuracy:

Epoch 970/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.1562 - accuracy: 0.9445 - val_loss: 0.3142 - val_accuracy: 0.9029
Epoch 971/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1548 - accuracy: 0.9455 - val_loss: 0.3170 - val_accuracy: 0.9044
Epoch 972/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1570 - accuracy: 0.9442 - val_loss: 0.3184 - val_accuracy: 0.9052
Epoch 973/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1584 - accuracy: 0.9437 - val_loss: 0.3139 - val_accuracy: 0.9052
Epoch 974/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1561 - accuracy: 0.9447 - val_loss: 0.3189 - val_accuracy: 0.9034
Epoch 975/1000
4/4 [==============================] - 1s 309ms/step - loss: 0.1590 - accuracy: 0.9435 - val_loss: 0.3236 - val_accuracy: 0.9027
Epoch 976/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1607 - accuracy: 0.9430 - val_loss: 0.3197 - val_accuracy:

In [7]:
model.save("../models/cnn1")

np.save('../models/cnn1_train_loss.npy', np.array(history.history['loss']))
np.save('../models/cnn1_test_loss.npy', np.array(history.history['val_loss']))
np.save('../models/cnn1_train_acc.npy', np.array(history.history['accuracy']))
np.save('../models/cnn1_test_acc.npy', np.array(history.history['val_accuracy']))

INFO:tensorflow:Assets written to: ../models/cnn1/assets


## Model 1 Testing

In [8]:
yhat = model.predict(Xtest)
yhat = np.array(np.argmax(yhat, axis=1))
acc = accuracy_score(ytest, yhat)
f1 = f1_score(ytest, yhat, average='macro')
print(f'Accuracy is {acc}')
print(f'F1 score is {f1}')

Accuracy is 0.9037980769230769
F1 score is 0.9036965109062519


## Model 2: Going deeper with dropout

In [10]:
np.random.seed(4622)
tf.random.set_seed(4622)

img_input = layers.Input(shape=(28,28,1))

x = layers.Conv2D(32, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)

x = layers.Dense(20, activation='relu')(x)
x = layers.Dropout(0.4)(x)

output = layers.Dense(26, activation='softmax')(x)

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss, metrics=['accuracy'])

In [11]:
history = model.fit(X, y, epochs=1000, batch_size=31200, validation_data=(Xtest, ytest))

Epoch 1/1000
4/4 [==============================] - 19s 4s/step - loss: 3.2560 - accuracy: 0.0418 - val_loss: 3.2484 - val_accuracy: 0.0546
Epoch 2/1000
4/4 [==============================] - 13s 3s/step - loss: 3.2413 - accuracy: 0.0561 - val_loss: 3.2301 - val_accuracy: 0.0668
Epoch 3/1000
4/4 [==============================] - 13s 3s/step - loss: 3.2129 - accuracy: 0.0673 - val_loss: 3.1710 - val_accuracy: 0.1151
Epoch 4/1000
4/4 [==============================] - 13s 3s/step - loss: 3.1647 - accuracy: 0.0781 - val_loss: 3.0886 - val_accuracy: 0.1572
Epoch 5/1000
4/4 [==============================] - 13s 3s/step - loss: 3.1093 - accuracy: 0.0914 - val_loss: 3.0291 - val_accuracy: 0.1656
Epoch 6/1000
4/4 [==============================] - 13s 3s/step - loss: 3.0521 - accuracy: 0.1015 - val_loss: 2.9083 - val_accuracy: 0.1787
Epoch 7/1000
4/4 [==============================] - 13s 3s/step - loss: 2.9787 - accuracy: 0.1123 - val_loss: 2.7752 - val_accuracy: 0.2133
Epoch 8/1000
4/4 [==

4/4 [==============================] - 13s 3s/step - loss: 0.3827 - accuracy: 0.8744 - val_loss: 0.1962 - val_accuracy: 0.9469
Epoch 118/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3774 - accuracy: 0.8767 - val_loss: 0.1937 - val_accuracy: 0.9474
Epoch 119/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3807 - accuracy: 0.8751 - val_loss: 0.1944 - val_accuracy: 0.9477
Epoch 120/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3744 - accuracy: 0.8771 - val_loss: 0.1958 - val_accuracy: 0.9480
Epoch 121/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3722 - accuracy: 0.8767 - val_loss: 0.1980 - val_accuracy: 0.9471
Epoch 122/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3717 - accuracy: 0.8774 - val_loss: 0.1934 - val_accuracy: 0.9476
Epoch 123/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3720 - accuracy: 0.8783 - val_loss: 0.1930 - val_accuracy: 0.9470
Epoch 124/1000
4/4 [=

Epoch 175/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3238 - accuracy: 0.8909 - val_loss: 0.1995 - val_accuracy: 0.9485
Epoch 176/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3187 - accuracy: 0.8913 - val_loss: 0.1998 - val_accuracy: 0.9485
Epoch 177/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3197 - accuracy: 0.8920 - val_loss: 0.1946 - val_accuracy: 0.9483
Epoch 178/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3231 - accuracy: 0.8898 - val_loss: 0.2009 - val_accuracy: 0.9484
Epoch 179/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3171 - accuracy: 0.8923 - val_loss: 0.1958 - val_accuracy: 0.9481
Epoch 180/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3128 - accuracy: 0.8926 - val_loss: 0.1981 - val_accuracy: 0.9479
Epoch 181/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3186 - accuracy: 0.8913 - val_loss: 0.1980 - val_accuracy: 0.9474
Epoch 

Epoch 233/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2876 - accuracy: 0.9003 - val_loss: 0.2082 - val_accuracy: 0.9488
Epoch 234/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2862 - accuracy: 0.9006 - val_loss: 0.2030 - val_accuracy: 0.9492
Epoch 235/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2860 - accuracy: 0.9013 - val_loss: 0.2083 - val_accuracy: 0.9496
Epoch 236/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2832 - accuracy: 0.9018 - val_loss: 0.2080 - val_accuracy: 0.9493
Epoch 237/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2823 - accuracy: 0.9022 - val_loss: 0.2083 - val_accuracy: 0.9495
Epoch 238/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2849 - accuracy: 0.9018 - val_loss: 0.2053 - val_accuracy: 0.9491
Epoch 239/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2837 - accuracy: 0.9018 - val_loss: 0.2095 - val_accuracy: 0.9499
Epoch 

Epoch 291/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2656 - accuracy: 0.9060 - val_loss: 0.2183 - val_accuracy: 0.9488
Epoch 292/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2653 - accuracy: 0.9065 - val_loss: 0.2166 - val_accuracy: 0.9486
Epoch 293/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2589 - accuracy: 0.9080 - val_loss: 0.2196 - val_accuracy: 0.9491
Epoch 294/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2603 - accuracy: 0.9075 - val_loss: 0.2200 - val_accuracy: 0.9479
Epoch 295/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2620 - accuracy: 0.9077 - val_loss: 0.2213 - val_accuracy: 0.9488
Epoch 296/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2635 - accuracy: 0.9071 - val_loss: 0.2214 - val_accuracy: 0.9486
Epoch 297/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2630 - accuracy: 0.9077 - val_loss: 0.2222 - val_accuracy: 0.9491
Epoch 

...

Epoch 755/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1959 - accuracy: 0.9290 - val_loss: 0.2913 - val_accuracy: 0.9505
Epoch 756/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1958 - accuracy: 0.9287 - val_loss: 0.2914 - val_accuracy: 0.9502
Epoch 757/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1941 - accuracy: 0.9299 - val_loss: 0.2900 - val_accuracy: 0.9499
Epoch 758/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1941 - accuracy: 0.9302 - val_loss: 0.2894 - val_accuracy: 0.9499
Epoch 759/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1936 - accuracy: 0.9293 - val_loss: 0.2962 - val_accuracy: 0.9497
Epoch 760/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1940 - accuracy: 0.9280 - val_loss: 0.2982 - val_accuracy: 0.9496
Epoch 761/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1945 - accuracy: 0.9285 - val_loss: 0.2883 - val_accuracy: 0.9500
Epoch 

Epoch 813/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1913 - accuracy: 0.9306 - val_loss: 0.2967 - val_accuracy: 0.9498
Epoch 814/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1934 - accuracy: 0.9294 - val_loss: 0.2982 - val_accuracy: 0.9492
Epoch 815/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1911 - accuracy: 0.9299 - val_loss: 0.3011 - val_accuracy: 0.9495
Epoch 816/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1922 - accuracy: 0.9303 - val_loss: 0.2994 - val_accuracy: 0.9497
Epoch 817/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1898 - accuracy: 0.9307 - val_loss: 0.3054 - val_accuracy: 0.9491
Epoch 818/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1912 - accuracy: 0.9298 - val_loss: 0.3053 - val_accuracy: 0.9492
Epoch 819/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1916 - accuracy: 0.9311 - val_loss: 0.2913 - val_accuracy: 0.9490
Epoch 

Epoch 871/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1865 - accuracy: 0.9326 - val_loss: 0.3058 - val_accuracy: 0.9501
Epoch 872/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1869 - accuracy: 0.9320 - val_loss: 0.3021 - val_accuracy: 0.9499
Epoch 873/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1878 - accuracy: 0.9322 - val_loss: 0.3032 - val_accuracy: 0.9502
Epoch 874/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1839 - accuracy: 0.9329 - val_loss: 0.3117 - val_accuracy: 0.9504
Epoch 875/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1860 - accuracy: 0.9325 - val_loss: 0.3104 - val_accuracy: 0.9496
Epoch 876/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1848 - accuracy: 0.9327 - val_loss: 0.3031 - val_accuracy: 0.9501
Epoch 877/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1868 - accuracy: 0.9323 - val_loss: 0.3040 - val_accuracy: 0.9504
Epoch 

Epoch 929/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1849 - accuracy: 0.9324 - val_loss: 0.3025 - val_accuracy: 0.9513
Epoch 930/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1840 - accuracy: 0.9324 - val_loss: 0.3113 - val_accuracy: 0.9508
Epoch 931/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1812 - accuracy: 0.9346 - val_loss: 0.3061 - val_accuracy: 0.9503
Epoch 932/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1831 - accuracy: 0.9330 - val_loss: 0.3107 - val_accuracy: 0.9501
Epoch 933/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1844 - accuracy: 0.9340 - val_loss: 0.3156 - val_accuracy: 0.9506
Epoch 934/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1832 - accuracy: 0.9337 - val_loss: 0.3106 - val_accuracy: 0.9501
Epoch 935/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1831 - accuracy: 0.9339 - val_loss: 0.3089 - val_accuracy: 0.9502
Epoch 

Epoch 987/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1813 - accuracy: 0.9352 - val_loss: 0.3163 - val_accuracy: 0.9494
Epoch 988/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1836 - accuracy: 0.9344 - val_loss: 0.3205 - val_accuracy: 0.9495
Epoch 989/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1811 - accuracy: 0.9350 - val_loss: 0.3214 - val_accuracy: 0.9499
Epoch 990/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1818 - accuracy: 0.9346 - val_loss: 0.3209 - val_accuracy: 0.9493
Epoch 991/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1785 - accuracy: 0.9348 - val_loss: 0.3233 - val_accuracy: 0.9490
Epoch 992/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1821 - accuracy: 0.9342 - val_loss: 0.3233 - val_accuracy: 0.9489
Epoch 993/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1810 - accuracy: 0.9353 - val_loss: 0.3210 - val_accuracy: 0.9488
Epoch 

In [13]:
model.save("../models/cnn2")

np.save('../models/cnn2_train_loss.npy', np.array(history.history['loss']))
np.save('../models/cnn2_test_loss.npy', np.array(history.history['val_loss']))
np.save('../models/cnn2_train_acc.npy', np.array(history.history['accuracy']))
np.save('../models/cnn2_test_acc.npy', np.array(history.history['val_accuracy']))

INFO:tensorflow:Assets written to: ../models/cnn2/assets


## Model 2 Testing

In [ ]:
yhat = model.predict(Xtest)
yhat = np.array(np.argmax(yhat, axis=1))
acc = accuracy_score(ytest, yhat)
f1 = f1_score(ytest, yhat, average='macro')
print(f'Accuracy is {acc}')
print(f'F1 score is {f1}')

## Model 3 - Model 2 + FR2

In [4]:
fr2_Xtrain = np.load(os.getcwd()+'/../feature_representations/feature_representation_2_train.npy')
fr2_Xtest = np.load(os.getcwd()+'/../feature_representations/feature_representation_2_test.npy')

X_flat_dim = X.shape[1]*X.shape[2]*X.shape[3]
dim2 = X_flat_dim + fr2_Xtrain.shape[1]

tmp_train = X.reshape(X.shape[0], X_flat_dim)
tmp_test = Xtest.reshape(Xtest.shape[0], X_flat_dim)

X = np.zeros((X.shape[0], dim2))
Xtest = np.zeros((Xtest.shape[0], dim2))

X[:, :X_flat_dim] = tmp_train
X[:, X_flat_dim:] = fr2_Xtrain
Xtest[:, :X_flat_dim] = tmp_test
Xtest[:, X_flat_dim:] = fr2_Xtest

# X = np.zeros(X.flatten().size + fr2_Xtrain.shape[0]) ???

In [9]:
np.random.seed(4622)
tf.random.set_seed(4622)

def get_img(tensor):
    return tensor[:,:784]

def get_vec(tensor):
    return tensor[:,784:]

img_input = layers.Input(shape=(801,))

# Lambda layers to split between original image (x) and FR2 (vec)
x = layers.Lambda(get_img)(img_input)
vec = layers.Lambda(get_vec)(img_input)
# Reshape layer for conv2d
x = layers.Reshape((28,28,1))(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)

x = layers.Dense(20, activation='relu')(x)
# Concatenate FR2 onto the 20 dim embedded space
x = layers.Concatenate(axis=-1)([x,vec])
x = layers.Dropout(0.4)(x)

output = layers.Dense(26, activation='softmax')(x)

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss, metrics=['accuracy'])

(None, 37)


In [6]:
history = model.fit(X, y, epochs=1000, batch_size=31200, validation_data=(Xtest, ytest))

Epoch 1/1000
4/4 [==============================] - 32s 5s/step - loss: 3.6063 - accuracy: 0.0449 - val_loss: 3.4074 - val_accuracy: 0.0546
Epoch 2/1000
4/4 [==============================] - 13s 3s/step - loss: 3.5407 - accuracy: 0.0536 - val_loss: 3.3215 - val_accuracy: 0.0712
Epoch 3/1000
4/4 [==============================] - 13s 3s/step - loss: 3.4741 - accuracy: 0.0636 - val_loss: 3.2544 - val_accuracy: 0.0832
Epoch 4/1000
4/4 [==============================] - 13s 3s/step - loss: 3.3998 - accuracy: 0.0771 - val_loss: 3.1521 - val_accuracy: 0.1047
Epoch 5/1000
4/4 [==============================] - 13s 3s/step - loss: 3.3120 - accuracy: 0.0940 - val_loss: 3.0126 - val_accuracy: 0.1504
Epoch 6/1000
4/4 [==============================] - 13s 3s/step - loss: 3.2295 - accuracy: 0.1144 - val_loss: 2.9009 - val_accuracy: 0.1956
Epoch 7/1000
4/4 [==============================] - 13s 3s/step - loss: 3.1455 - accuracy: 0.1329 - val_loss: 2.8325 - val_accuracy: 0.2136
Epoch 8/1000
4/4 [==

4/4 [==============================] - 13s 3s/step - loss: 0.3486 - accuracy: 0.8886 - val_loss: 0.1886 - val_accuracy: 0.9449
Epoch 118/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3492 - accuracy: 0.8907 - val_loss: 0.1880 - val_accuracy: 0.9456
Epoch 119/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3479 - accuracy: 0.8902 - val_loss: 0.1914 - val_accuracy: 0.9456
Epoch 120/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3468 - accuracy: 0.8918 - val_loss: 0.1893 - val_accuracy: 0.9450
Epoch 121/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3396 - accuracy: 0.8919 - val_loss: 0.1896 - val_accuracy: 0.9458
Epoch 122/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3431 - accuracy: 0.8910 - val_loss: 0.1910 - val_accuracy: 0.9460
Epoch 123/1000
4/4 [==============================] - 13s 3s/step - loss: 0.3377 - accuracy: 0.8938 - val_loss: 0.1881 - val_accuracy: 0.9450
Epoch 124/1000
4/4 [=

Epoch 175/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2850 - accuracy: 0.9078 - val_loss: 0.1885 - val_accuracy: 0.9486
Epoch 176/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2827 - accuracy: 0.9082 - val_loss: 0.1935 - val_accuracy: 0.9488
Epoch 177/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2883 - accuracy: 0.9076 - val_loss: 0.1893 - val_accuracy: 0.9483
Epoch 178/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2856 - accuracy: 0.9083 - val_loss: 0.1852 - val_accuracy: 0.9481
Epoch 179/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2858 - accuracy: 0.9079 - val_loss: 0.1894 - val_accuracy: 0.9489
Epoch 180/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2870 - accuracy: 0.9090 - val_loss: 0.1858 - val_accuracy: 0.9483
Epoch 181/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2796 - accuracy: 0.9090 - val_loss: 0.1906 - val_accuracy: 0.9476
Epoch 

Epoch 233/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2520 - accuracy: 0.9163 - val_loss: 0.1928 - val_accuracy: 0.9489
Epoch 234/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2508 - accuracy: 0.9176 - val_loss: 0.1919 - val_accuracy: 0.9494
Epoch 235/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2488 - accuracy: 0.9179 - val_loss: 0.1925 - val_accuracy: 0.9490
Epoch 236/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2498 - accuracy: 0.9183 - val_loss: 0.1943 - val_accuracy: 0.9490
Epoch 237/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2447 - accuracy: 0.9191 - val_loss: 0.1942 - val_accuracy: 0.9495
Epoch 238/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2476 - accuracy: 0.9177 - val_loss: 0.1943 - val_accuracy: 0.9488
Epoch 239/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2452 - accuracy: 0.9198 - val_loss: 0.1958 - val_accuracy: 0.9492
Epoch 

Epoch 291/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2251 - accuracy: 0.9244 - val_loss: 0.2046 - val_accuracy: 0.9486
Epoch 292/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2241 - accuracy: 0.9248 - val_loss: 0.1973 - val_accuracy: 0.9499
Epoch 293/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2260 - accuracy: 0.9244 - val_loss: 0.1997 - val_accuracy: 0.9494
Epoch 294/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2273 - accuracy: 0.9239 - val_loss: 0.1975 - val_accuracy: 0.9494
Epoch 295/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2260 - accuracy: 0.9247 - val_loss: 0.2021 - val_accuracy: 0.9487
Epoch 296/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2242 - accuracy: 0.9240 - val_loss: 0.2017 - val_accuracy: 0.9493
Epoch 297/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2263 - accuracy: 0.9245 - val_loss: 0.2048 - val_accuracy: 0.9490
Epoch 

...

Epoch 755/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1596 - accuracy: 0.9434 - val_loss: 0.2575 - val_accuracy: 0.9500
Epoch 756/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1581 - accuracy: 0.9445 - val_loss: 0.2546 - val_accuracy: 0.9504
Epoch 757/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1568 - accuracy: 0.9434 - val_loss: 0.2588 - val_accuracy: 0.9503
Epoch 758/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1574 - accuracy: 0.9437 - val_loss: 0.2588 - val_accuracy: 0.9503
Epoch 759/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1570 - accuracy: 0.9435 - val_loss: 0.2548 - val_accuracy: 0.9504
Epoch 760/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1579 - accuracy: 0.9440 - val_loss: 0.2593 - val_accuracy: 0.9502
Epoch 761/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1576 - accuracy: 0.9438 - val_loss: 0.2601 - val_accuracy: 0.9508
Epoch 

Epoch 813/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1555 - accuracy: 0.9448 - val_loss: 0.2639 - val_accuracy: 0.9494
Epoch 814/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1555 - accuracy: 0.9450 - val_loss: 0.2559 - val_accuracy: 0.9495
Epoch 815/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1541 - accuracy: 0.9452 - val_loss: 0.2635 - val_accuracy: 0.9493
Epoch 816/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1542 - accuracy: 0.9452 - val_loss: 0.2625 - val_accuracy: 0.9500
Epoch 817/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1537 - accuracy: 0.9451 - val_loss: 0.2595 - val_accuracy: 0.9497
Epoch 818/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1523 - accuracy: 0.9458 - val_loss: 0.2679 - val_accuracy: 0.9492
Epoch 819/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1550 - accuracy: 0.9451 - val_loss: 0.2659 - val_accuracy: 0.9495
Epoch 

Epoch 871/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1531 - accuracy: 0.9453 - val_loss: 0.2619 - val_accuracy: 0.9504
Epoch 872/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1524 - accuracy: 0.9449 - val_loss: 0.2648 - val_accuracy: 0.9499
Epoch 873/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1549 - accuracy: 0.9453 - val_loss: 0.2636 - val_accuracy: 0.9500
Epoch 874/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1521 - accuracy: 0.9465 - val_loss: 0.2649 - val_accuracy: 0.9493
Epoch 875/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1516 - accuracy: 0.9460 - val_loss: 0.2623 - val_accuracy: 0.9492
Epoch 876/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1519 - accuracy: 0.9454 - val_loss: 0.2644 - val_accuracy: 0.9493
Epoch 877/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1511 - accuracy: 0.9462 - val_loss: 0.2654 - val_accuracy: 0.9492
Epoch 

Epoch 929/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1479 - accuracy: 0.9473 - val_loss: 0.2693 - val_accuracy: 0.9502
Epoch 930/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1494 - accuracy: 0.9465 - val_loss: 0.2729 - val_accuracy: 0.9498
Epoch 931/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1481 - accuracy: 0.9463 - val_loss: 0.2716 - val_accuracy: 0.9500
Epoch 932/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1450 - accuracy: 0.9474 - val_loss: 0.2758 - val_accuracy: 0.9497
Epoch 933/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1468 - accuracy: 0.9475 - val_loss: 0.2768 - val_accuracy: 0.9496
Epoch 934/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1477 - accuracy: 0.9472 - val_loss: 0.2739 - val_accuracy: 0.9500
Epoch 935/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1474 - accuracy: 0.9471 - val_loss: 0.2733 - val_accuracy: 0.9497
Epoch 

Epoch 987/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1479 - accuracy: 0.9466 - val_loss: 0.2758 - val_accuracy: 0.9494
Epoch 988/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1450 - accuracy: 0.9481 - val_loss: 0.2739 - val_accuracy: 0.9495
Epoch 989/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1475 - accuracy: 0.9470 - val_loss: 0.2783 - val_accuracy: 0.9495
Epoch 990/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1456 - accuracy: 0.9479 - val_loss: 0.2790 - val_accuracy: 0.9502
Epoch 991/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1446 - accuracy: 0.9484 - val_loss: 0.2768 - val_accuracy: 0.9503
Epoch 992/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1478 - accuracy: 0.9470 - val_loss: 0.2722 - val_accuracy: 0.9499
Epoch 993/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1460 - accuracy: 0.9480 - val_loss: 0.2713 - val_accuracy: 0.9503
Epoch 

In [7]:
model.save("../models/cnn3")

np.save('../models/cnn3_train_loss.npy', np.array(history.history['loss']))
np.save('../models/cnn3_test_loss.npy', np.array(history.history['val_loss']))
np.save('../models/cnn3_train_acc.npy', np.array(history.history['accuracy']))
np.save('../models/cnn3_test_acc.npy', np.array(history.history['val_accuracy']))

INFO:tensorflow:Assets written to: ../models/cnn3/assets


## Model 3 Testing

In [8]:
yhat = model.predict(Xtest)
yhat = np.array(np.argmax(yhat, axis=1))
acc = accuracy_score(ytest, yhat)
f1 = f1_score(ytest, yhat, average='macro')
print(f'Accuracy is {acc}')
print(f'F1 score is {f1}')

Accuracy is 0.9508653846153846
F1 score is 0.9509203079552352


## Model 4: 37 length embedded space

In [6]:
np.random.seed(4622)
tf.random.set_seed(4622)

img_input = layers.Input(shape=(28,28,1))

x = layers.Conv2D(32, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)

x = layers.Dense(37, activation='relu')(x)
x = layers.Dropout(0.4)(x)

output = layers.Dense(26, activation='softmax')(x)

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss, metrics=['accuracy'])

In [7]:
history = model.fit(X, y, epochs=1000, batch_size=31200, validation_data=(Xtest, ytest))

Epoch 1/1000
4/4 [==============================] - 32s 5s/step - loss: 3.2531 - accuracy: 0.0421 - val_loss: 3.2388 - val_accuracy: 0.0884
Epoch 2/1000
4/4 [==============================] - 13s 3s/step - loss: 3.1876 - accuracy: 0.0708 - val_loss: 3.0790 - val_accuracy: 0.1435
Epoch 3/1000
4/4 [==============================] - 13s 3s/step - loss: 3.0918 - accuracy: 0.0993 - val_loss: 2.9639 - val_accuracy: 0.1860
Epoch 4/1000
4/4 [==============================] - 13s 3s/step - loss: 2.9631 - accuracy: 0.1388 - val_loss: 2.7697 - val_accuracy: 0.2961
Epoch 5/1000
4/4 [==============================] - 13s 3s/step - loss: 2.8206 - accuracy: 0.1758 - val_loss: 2.5357 - val_accuracy: 0.3593
Epoch 6/1000
4/4 [==============================] - 13s 3s/step - loss: 2.6676 - accuracy: 0.2241 - val_loss: 2.3075 - val_accuracy: 0.4319
Epoch 7/1000
4/4 [==============================] - 13s 3s/step - loss: 2.4933 - accuracy: 0.2703 - val_loss: 2.0559 - val_accuracy: 0.5121
Epoch 8/1000
4/4 [==

4/4 [==============================] - 13s 3s/step - loss: 0.2159 - accuracy: 0.9306 - val_loss: 0.1711 - val_accuracy: 0.9470
Epoch 118/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2155 - accuracy: 0.9314 - val_loss: 0.1683 - val_accuracy: 0.9481
Epoch 119/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2149 - accuracy: 0.9317 - val_loss: 0.1694 - val_accuracy: 0.9476
Epoch 120/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2118 - accuracy: 0.9320 - val_loss: 0.1708 - val_accuracy: 0.9475
Epoch 121/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2125 - accuracy: 0.9323 - val_loss: 0.1700 - val_accuracy: 0.9473
Epoch 122/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2083 - accuracy: 0.9332 - val_loss: 0.1699 - val_accuracy: 0.9467
Epoch 123/1000
4/4 [==============================] - 13s 3s/step - loss: 0.2089 - accuracy: 0.9327 - val_loss: 0.1689 - val_accuracy: 0.9488
Epoch 124/1000
4/4 [=

Epoch 175/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1733 - accuracy: 0.9411 - val_loss: 0.1719 - val_accuracy: 0.9481
Epoch 176/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1750 - accuracy: 0.9414 - val_loss: 0.1700 - val_accuracy: 0.9472
Epoch 177/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1742 - accuracy: 0.9421 - val_loss: 0.1687 - val_accuracy: 0.9479
Epoch 178/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1720 - accuracy: 0.9419 - val_loss: 0.1701 - val_accuracy: 0.9475
Epoch 179/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1734 - accuracy: 0.9419 - val_loss: 0.1695 - val_accuracy: 0.9475
Epoch 180/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1743 - accuracy: 0.9420 - val_loss: 0.1691 - val_accuracy: 0.9482
Epoch 181/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1725 - accuracy: 0.9420 - val_loss: 0.1694 - val_accuracy: 0.9476
Epoch 

Epoch 233/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1508 - accuracy: 0.9477 - val_loss: 0.1738 - val_accuracy: 0.9496
Epoch 234/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1491 - accuracy: 0.9478 - val_loss: 0.1782 - val_accuracy: 0.9476
Epoch 235/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1486 - accuracy: 0.9485 - val_loss: 0.1731 - val_accuracy: 0.9491
Epoch 236/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1476 - accuracy: 0.9481 - val_loss: 0.1761 - val_accuracy: 0.9491
Epoch 237/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1481 - accuracy: 0.9488 - val_loss: 0.1765 - val_accuracy: 0.9496
Epoch 238/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1489 - accuracy: 0.9477 - val_loss: 0.1747 - val_accuracy: 0.9492
Epoch 239/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1454 - accuracy: 0.9494 - val_loss: 0.1767 - val_accuracy: 0.9493
Epoch 

Epoch 291/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1343 - accuracy: 0.9516 - val_loss: 0.1771 - val_accuracy: 0.9503
Epoch 292/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1329 - accuracy: 0.9518 - val_loss: 0.1804 - val_accuracy: 0.9494
Epoch 293/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1340 - accuracy: 0.9513 - val_loss: 0.1787 - val_accuracy: 0.9498
Epoch 294/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1336 - accuracy: 0.9517 - val_loss: 0.1776 - val_accuracy: 0.9487
Epoch 295/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1326 - accuracy: 0.9519 - val_loss: 0.1819 - val_accuracy: 0.9500
Epoch 296/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1348 - accuracy: 0.9519 - val_loss: 0.1777 - val_accuracy: 0.9498
Epoch 297/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1329 - accuracy: 0.9528 - val_loss: 0.1792 - val_accuracy: 0.9496
Epoch 

...

Epoch 755/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0887 - accuracy: 0.9653 - val_loss: 0.2170 - val_accuracy: 0.9495
Epoch 756/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0883 - accuracy: 0.9656 - val_loss: 0.2180 - val_accuracy: 0.9503
Epoch 757/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0888 - accuracy: 0.9651 - val_loss: 0.2154 - val_accuracy: 0.9501
Epoch 758/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0894 - accuracy: 0.9649 - val_loss: 0.2162 - val_accuracy: 0.9501
Epoch 759/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0900 - accuracy: 0.9649 - val_loss: 0.2157 - val_accuracy: 0.9498
Epoch 760/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0883 - accuracy: 0.9654 - val_loss: 0.2124 - val_accuracy: 0.9502
Epoch 761/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0881 - accuracy: 0.9662 - val_loss: 0.2174 - val_accuracy: 0.9495
Epoch 

Epoch 813/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0872 - accuracy: 0.9665 - val_loss: 0.2147 - val_accuracy: 0.9504
Epoch 814/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0878 - accuracy: 0.9661 - val_loss: 0.2160 - val_accuracy: 0.9505
Epoch 815/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0869 - accuracy: 0.9660 - val_loss: 0.2179 - val_accuracy: 0.9502
Epoch 816/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0850 - accuracy: 0.9665 - val_loss: 0.2187 - val_accuracy: 0.9500
Epoch 817/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0858 - accuracy: 0.9666 - val_loss: 0.2223 - val_accuracy: 0.9501
Epoch 818/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0848 - accuracy: 0.9668 - val_loss: 0.2210 - val_accuracy: 0.9503
Epoch 819/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0873 - accuracy: 0.9660 - val_loss: 0.2201 - val_accuracy: 0.9505
Epoch 

Epoch 871/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0846 - accuracy: 0.9671 - val_loss: 0.2189 - val_accuracy: 0.9498
Epoch 872/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0857 - accuracy: 0.9667 - val_loss: 0.2211 - val_accuracy: 0.9496
Epoch 873/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0848 - accuracy: 0.9672 - val_loss: 0.2211 - val_accuracy: 0.9492
Epoch 874/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0836 - accuracy: 0.9673 - val_loss: 0.2190 - val_accuracy: 0.9498
Epoch 875/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0839 - accuracy: 0.9673 - val_loss: 0.2198 - val_accuracy: 0.9498
Epoch 876/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0827 - accuracy: 0.9674 - val_loss: 0.2260 - val_accuracy: 0.9490
Epoch 877/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0847 - accuracy: 0.9669 - val_loss: 0.2218 - val_accuracy: 0.9489
Epoch 

Epoch 929/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0817 - accuracy: 0.9676 - val_loss: 0.2247 - val_accuracy: 0.9493
Epoch 930/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0824 - accuracy: 0.9677 - val_loss: 0.2260 - val_accuracy: 0.9497
Epoch 931/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0826 - accuracy: 0.9677 - val_loss: 0.2257 - val_accuracy: 0.9501
Epoch 932/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0802 - accuracy: 0.9679 - val_loss: 0.2295 - val_accuracy: 0.9496
Epoch 933/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0829 - accuracy: 0.9678 - val_loss: 0.2243 - val_accuracy: 0.9498
Epoch 934/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0820 - accuracy: 0.9676 - val_loss: 0.2259 - val_accuracy: 0.9503
Epoch 935/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0823 - accuracy: 0.9679 - val_loss: 0.2306 - val_accuracy: 0.9495
Epoch 

Epoch 987/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0806 - accuracy: 0.9687 - val_loss: 0.2279 - val_accuracy: 0.9498
Epoch 988/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0824 - accuracy: 0.9676 - val_loss: 0.2248 - val_accuracy: 0.9493
Epoch 989/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0805 - accuracy: 0.9688 - val_loss: 0.2269 - val_accuracy: 0.9488
Epoch 990/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0810 - accuracy: 0.9681 - val_loss: 0.2261 - val_accuracy: 0.9497
Epoch 991/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0806 - accuracy: 0.9688 - val_loss: 0.2280 - val_accuracy: 0.9494
Epoch 992/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0815 - accuracy: 0.9682 - val_loss: 0.2263 - val_accuracy: 0.9495
Epoch 993/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0810 - accuracy: 0.9682 - val_loss: 0.2297 - val_accuracy: 0.9494
Epoch 

In [8]:
model.save("../models/cnn4")

np.save('../models/cnn4_train_loss.npy', np.array(history.history['loss']))
np.save('../models/cnn4_test_loss.npy', np.array(history.history['val_loss']))
np.save('../models/cnn4_train_acc.npy', np.array(history.history['accuracy']))
np.save('../models/cnn4_test_acc.npy', np.array(history.history['val_accuracy']))

INFO:tensorflow:Assets written to: ../models/cnn4/assets


## Model 4 Testing

In [9]:
yhat = model.predict(Xtest)
yhat = np.array(np.argmax(yhat, axis=1))
acc = accuracy_score(ytest, yhat)
f1 = f1_score(ytest, yhat, average='macro')
print(f'Accuracy is {acc}')
print(f'F1 score is {f1}')

Accuracy is 0.9498076923076924
F1 score is 0.9498531301738868
